In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [20]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
import keras.utils as np_utils

# Define train and test paths
train_path = 'drive/MyDrive/DevanagariHandwrittenDigitDataset/Train'
test_path = 'drive/MyDrive/DevanagariHandwrittenDigitDataset/Test'

# Define the number of classes
num_classes = 10

# Define image size and number of channels
img_rows, img_cols, img_channels = 32, 32, 1

In [3]:
# Define a function to load the data from the specified path
import re

def load_data(path):
    data = []
    labels = []
    pattern = r"\d+" # regular expression to match digits in folder name
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_rows, img_cols))
            data.append(image)
            match = re.search(pattern, folder)
            labels.append(int(match.group())) # convert matched digits to integer label
    data = np.array(data)
    data = data.reshape(data.shape[0], img_rows, img_cols, img_channels)
    data = data.astype('float32')
    data /= 255
    labels = np.array(labels)
    labels = np_utils.to_categorical(labels, num_classes)
    return data, labels

In [4]:
# Load the training data
X_train, y_train = load_data(train_path)

# Load the testing data
X_test, y_test = load_data(test_path)

In [26]:
# Define the CNN-LSTM model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, img_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
from keras.layers import Reshape
from keras.layers import TimeDistributed, LSTM, Dense
model.add(Flatten())
model.add(Reshape((8, 8, 64)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(128)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
callback = EarlyStopping(monitor="loss", patience=2)

# Fit the model to the training data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, callbacks=[callback])

Epoch 1/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 72s 186ms/step - accuracy: 0.6214 - loss: 1.0630 - val_accuracy: 0.9690 - val_loss: 0.1106
Epoch 2/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 75s 159ms/step - accuracy: 0.9713 - loss: 0.0950 - val_accuracy: 0.9840 - val_loss: 0.0578
Epoch 3/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 43s 161ms/step - accuracy: 0.9830 - loss: 0.0562 - val_accuracy: 0.9883 - val_loss: 0.0451
Epoch 4/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 44s 164ms/step - accuracy: 0.9910 - loss: 0.0318 - val_accuracy: 0.9887 - val_loss: 0.0379
Epoch 5/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - accuracy: 0.9956 - loss: 0.0163 - val_accuracy: 0.9883 - val_loss: 0.0395
Epoch 6/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - accuracy: 0.9917 - loss: 0.0285 - val_accuracy: 0.9877 - val_loss: 0.0400
Epoch 7/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 80s 158ms/step - accuracy: 0.9965 - loss: 0.0116 - val_accuracy: 0.9903 - val_loss: 0.0343
Epoch 8/20
266/266 ━━━━━━━━━━━━━━━━━━━━ 89s 185ms/step - accuracy: 0.9960 - loss: 0

In [29]:

# Evaluate the model on the testing data
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 99.00%
